In [1]:
import luigi
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/home/fbuonerba/codes/')
from mp_functions import upload_log_return, upload_factor_loadings
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
import calendar
import json
import urllib.request
import multiprocessing as mp
import sklearn
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [9]:
#####Returns: January to April#####

with open('/home/fbuonerba/codes/meta_data/top_coins.txt') as ff:
    coins=json.load(ff)
coins=list(coins)
####LIZA is outlier for 2 of the risk factors...####
coins.remove('LIZA')
quotes=['USD','BTC']
#sysm=[str(coin)+'_'+str(quote) for coin in coins for quote in quotes]
t=1515283200
matrix=[]
#matrix of daily log_returns (time, base_quote)
while t<=1522540801:
    ret_t=[]
    for base in coins:
        for quote in quotes:
            returns=upload_log_return(t, base, quote, 86400)
            if np.isnan(returns)==True:
                returns=0
            ret_t.append(returns)
            #print(base,quote)
    matrix.append(ret_t)
    t+=86400
R=np.array(matrix)

In [2]:
#####Returns: April 2nd to May 2nd#####

with open('/home/fbuonerba/codes/meta_data/top_coins.txt') as ff:
    coins=json.load(ff)
coins=list(coins)
####LIZA is outlier for 2 of the risk factors...####
coins.remove('LIZA')
quotes=['USD','BTC']
#sysm=[str(coin)+'_'+str(quote) for coin in coins for quote in quotes]
t=1522627200
matrix=[]
#matrix of daily log_returns (time, base_quote)
while t<=1525219201:
    ret_t=[]
    for base in coins:
        for quote in quotes:
            returns=upload_log_return(t, base, quote, 86400)
            if np.isnan(returns)==True:
                returns=0
            ret_t.append(returns)
            #print(base,quote)
    matrix.append(ret_t)
    t+=86400
R=np.array(matrix)

In [3]:
#####Factors: January to April#####

factors=[]
factor_names1=['variance_', 'strength_', 'high_low_']
factor_names2=['turnover_', 'log_mkcap_']
factor_names3=['coin_ratio_']
path='/home/fbuonerba/factor_loadings/'
finpath1='_'+str(1515283200)+'_'+str(1522540800)+'_'+str(604800)+'.txt'
finpath2='_'+str(1515283200)+'_'+str(1522540800)+'.txt'
finpath3='.txt'
for base in coins:
    for quote in quotes:
        sym=str(base)+'_'+str(quote)
        sym_row=[]
        for name in factor_names1:
            with open(path+name+sym+finpath1) as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names2:
            with open(path+name+sym+finpath2) as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names3:
            #symll=sym.split('_')
            with open(path+name+base+finpath3) as file:
                x=json.load(file)
            sym_row.append(x)
        #sym_row.append(1)
        factors.append(sym_row)
factors=np.array(factors)  
#normalization using z-score along coin axis:
factors=(factors-np.mean(factors,axis=0))/np.var(factors,axis=0)**.5

In [53]:
#regression against returns on April 1st only.
t=1522540800
ret=[]
for base in coins:
    for quote in quotes:
        returns=upload_log_return(t, base, quote, 86400)
        if np.isnan(returns)==True:
            returns=0
        ret.append(returns)
ret=np.array(ret)
reg=linear_model.LinearRegression()
reg.fit(factors,ret.T)
beta=reg.coef_
inter=reg.intercept_
ret_hat=reg.predict(factors)
rss=np.linalg.norm(ret-ret_hat)**2
var_error=rss/(factors.shape[0]-factors.shape[1])
tss=np.linalg.norm(ret-np.mean(ret))**2
r_square=1-rss/tss
F_score=(tss-rss)/(factors.shape[1]*var_error)
cov_beta=np.linalg.inv(np.dot(factors.T,factors))*var_error
std_errors=np.sqrt(cov_beta.diagonal())
t_stats=beta/std_errors

df=pd.DataFrame()
df['factor_name']=['returns_variance', 'returns_strength', 'returns_high_low', 'turnover', 'log_mkcap', 'coin_ratio']
df['beta']=beta
df['t_stats']=t_stats
df1=pd.DataFrame()
df1['r_squared']=np.array([r_square])
df1['F_score']=np.array([F_score])
print('Regression on April 1st returns.')
print('Definition of factors, for each coin pair:')
print('*returns_variance = var(returns_t), over weekly returns Jan 7th-Apr 1st.')
print('*returns_strength=sum_t( log(1+return_t) ) over returns as above.')
print('*returns_high_low=log( max_t(return_t)/min_t(return_t) ) over returns as above.')
print('*turnover=(total traded volume)/(average coin supply) over trades Jan 7th-Apr 1st.')
print('*log_mkcap=average_t( log(coin_supply_t*price_t) ) over weekly supply and price Jan 7th-Apr 1st.')
print('*coin_ratio=average_t(coin_supply_t/coin_supply_ever) over weekly supply as above.')
print('Factor loadings have been scaled by z-score.')
print('')
print(df)
print(df1)

Regression on April 1st returns.
Definition of factors, for each coin pair:
*returns_variance = var(returns_t), over weekly returns Jan 7th-Apr 1st.
*returns_strength=sum_t( log(1+return_t) ) over returns as above.
*returns_high_low=log( max_t(return_t)/min_t(return_t) ) over returns as above.
*turnover=(total traded volume)/(average coin supply) over trades Jan 7th-Apr 1st.
*log_mkcap=average_t( log(coin_supply_t*price_t) ) over weekly supply and price Jan 7th-Apr 1st.
*coin_ratio=average_t(coin_supply_t/coin_supply_ever) over weekly supply as above.
Factor loadings have been scaled by z-score.

        factor_name      beta   t_stats
0  returns_variance -0.001987 -0.450239
1  returns_strength  0.010160  1.797171
2  returns_high_low  0.005005  0.682049
3          turnover -0.000110 -0.031699
4         log_mkcap -0.010219 -1.937250
5        coin_ratio  0.005506  1.512765
   r_squared   F_score
0   0.190144  3.208763


In [40]:
print('This model is tested on coin returns on April 1st')
print('There are 88 coin pairs and 6 factors')
print('Factor names are:')
print('returns_variance, returns_strength, returns_high_low, turnover, log_mkcap, coin_ratio')
print('The dimensionality of various variables are listed below:')
print('factors size =', factors.shape)
print('returns size =', ret.shape)
print('beta size =', beta.shape)
print('beta =', beta)
print('intercept =', inter)
print('r_squared =', r_square)
print('F_score =', F_score)
print('t_stats =', t_stats)


This model is tested on coin returns on April 1st
There are 88 coin pairs and 6 factors
Factor names are:
returns_variance, returns_strength, returns_high_low, turnover, log_mkcap, coin_ratio
The dimensionality of various variables are listed below:
factors size = (88, 6)
returns size = (88,)
beta size = (6,)
beta = [-0.00198744  0.01016     0.00500487 -0.00011009 -0.01021864  0.00550614]
intercept = -0.02024262125528201
r_squared = 0.19014406720890709
F_score = 3.20876279344209
t_stats = [-0.45023932  1.79717082  0.68204906 -0.03169944 -1.93725015  1.51276472]


In [ ]:
#####February to May#####

from mp_functions import upload_log_return
with open('/home/fbuonerba/codes/meta_data/top_coins.txt') as ff:
    coins=json.load(ff)
coins=list(coins)
####LIZA is outlier for 2 of the risk factors...####
coins.remove('LIZA')
quotes=['USD','BTC']
#sysm=[str(coin)+'_'+str(quote) for coin in coins for quote in quotes]
t=1517702400
matrix=[]
#matrix of daily log_returns (time, base_quote)
while t<=1525564801:
    ret_t=[]
    for base in coins:
        for quote in quotes:
            returns=upload_log_return(t, base, quote, 86400)
            if np.isnan(returns)==True:
                returns=0
            ret_t.append(returns)
            #print(base,quote)
    matrix.append(ret_t)
    t+=86400
R=np.array(matrix)

factors=[]
factor_names1=['variance_', 'strength_', 'high_low_']
factor_names2=['turnover_', 'log_mkcap_']
factor_names3=['coin_ratio_']
path='/home/fbuonerba/codes/factor_loadings/'
finpath1='_'+str(1517702400)+'_'+str(1525564800)+'_'+str(604800)+'.txt'
finpath2='_'+str(1517702400)+'_'+str(1525564800)+'.txt'
finpath3='.txt'
for base in coins:
    for quote in quotes:
        sym=str(base)+'_'+str(quote)
        sym_row=[]
        for name in factor_names1:
            with open(path+name+sym+finpath1) as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names2:
            with open(path+name+sym+finpath2) as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names3:
            #symll=sym.split('_')
            with open(path+name+base+finpath3) as file:
                x=json.load(file)
            sym_row.append(x)
        #sym_row.append(1)
        factors.append(sym_row)
factors=np.array(factors)  
#normalization using z-score along coin axis:
factors=(factors-np.mean(factors,axis=0))/np.var(factors,axis=0)**.5
#X_with_ones = np.hstack((np.ones((factors.shape[0], 1)), factors))

In [ ]:
reg = linear_model.LinearRegression()
reg.fit(factors, R.T)
#reg.fit pre-processes factors - centers along axis=0
#adding a colums of ones to factors makes no difference, there will be intercept anyway.
beta_=reg.coef_
inter_=reg.intercept_

R_hat1=reg.predict(factors)
R_hat=np.dot(factors,beta_.T)+inter_
print('r2 using daily mean =',reg.score(factors,R.T), r2_score(R.T,R_hat1,multioutput='variance_weighted'))
print('r2 using total mean =',r2_score(R.T,R_hat),)
#reg.score computes the mean along axis=0
#r2_score computes the mean over full matrix


In [ ]:
#factors must have extra column of ones for this to work.
factors=np.hstack((np.ones((factors.shape[0], 1)), factors))
beta=np.linalg.lstsq(factors, R.T,rcond=None)
gamma=beta[0]
R_hat_np=np.dot(factors,gamma)
residuals=np.sum( (R.T-R_hat_np)**2)
#average returns over days, for each coin
mean=np.mean(R, axis=0)
sum_squares=np.sum((R-mean.T)**2)
r2_coin=1-residuals/sum_squares
#average returns over coins, for each day.
mean_bs=np.mean(R, axis=1)
sum_squares_bs=np.sum((R.T-mean_bs)**2)
r2_daily=1-residuals/sum_squares_bs
print('r2 using coin mean =',r2_coin)
print('r2 using daily mean =',r2_daily)
#just average over days and coins
print('r2 using total mean =',1-residuals/np.linalg.norm(R.T-np.mean(R))**2)

In [ ]:
######################################################

In [ ]:
#regression against each one single time, BTC/USD
factor_test=factors
r=[]
for i in range(85):
    R_test=R[i]
    beta_test=np.linalg.lstsq(factor_test,R_test.T, rcond=None)
    S=np.dot(factors,beta_test[0])
    res_test=np.linalg.norm(R_test-S)**2
    r.append(1-res_test/np.linalg.norm(R_test)**2)
r=np.array(r)
print(np.max(r), np.min(r))

In [ ]:
reg=linear_model.LinearRegression()
for i in range(6):
    fac=factors.T[i].reshape(-1,1)
    ret=R[80].T
    #print(fac.shape)
    reg.fit(fac,ret)
    temp_beta=reg.coef_[0]
    temp_inter=reg.intercept_
    print(temp_beta, temp_inter)
    print(reg.score(fac,ret))
    import matplotlib.pyplot as plt
    plt.plot(fac, ret,'o')
    plt.plot(fac, temp_beta*fac + temp_inter, 'r', label='Fitted line')
    plt.show()

In [ ]:
for t in range(88):
    print(coins[int(t/2)],quotes[t%2])
    plt.plot(range(len(R)), R.T[t])
    plt.show()

In [7]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [33]:
factors=sm.add_constant(factors)
X=factors
y=R.T
results = sm.OLS(y,X).fit()
results